<a href="https://colab.research.google.com/github/Dahlia-Chehata/RumourEval-with-VLLMs/blob/main/rumour_eval_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import openai
import pandas as pd
import json
import os

In [ ]:
cd /content/drive/MyDrive/CS\ 886\:\ Final\ Project

/content/drive/MyDrive/CS 886: Final Project


# 2. Pre-processing

## 2.1. Train and Dev

In [ ]:
!tar -xf semeval2017-task8-dataset.tar.bz2

In [ ]:
data_sources = [entry for entry in os.listdir('semeval2017-task8-dataset/rumoureval-data')]
data_sources

['ebola-essien',
 'putinmissing',
 'charliehebdo',
 'prince-toronto',
 'ferguson',
 'germanwings-crash',
 'ottawashooting',
 'sydneysiege']

In [ ]:
ground_truth_path = 'semeval2017-task8-dataset/traindev'
gt_trainA_path = os.path.join(ground_truth_path, 'rumoureval-subtaskA-train.json')
gt_devA_path = os.path.join(ground_truth_path, 'rumoureval-subtaskA-dev.json')
gt_trainB_path = os.path.join(ground_truth_path, 'rumoureval-subtaskB-train.json')
gt_devB_path = os.path.join(ground_truth_path, 'rumoureval-subtaskB-dev.json')

In [ ]:
with open(gt_trainA_path, 'r') as f:
    gt_trainA = json.load(f)

with open(gt_devA_path, 'r') as f:
    gt_devA = json.load(f)

with open(gt_trainB_path, 'r') as f:
    gt_trainB = json.load(f)

with open(gt_devB_path, 'r') as f:
    gt_devB = json.load(f)

In [ ]:
data_path = 'semeval2017-task8-dataset/rumoureval-data'

In [ ]:
listA_train = []
listA_dev = []

listB_train = []
listB_dev = []

for entry in os.listdir(data_path):
  data_source_path = os.path.join(data_path, entry)
  for subentry in os.listdir(data_source_path):
    tweet_path = os.path.join(data_source_path, subentry, "source-tweet")
    #print(tweet_path)
    for json_file in os.listdir(tweet_path):
      # print(json_file)
      json_file_path = os.path.join(tweet_path, json_file)
      data = pd.read_json(path_or_buf=json_file_path, lines=True)
      subentry = str(subentry)
      if subentry in gt_devB:
        listB_dev.append({'Journal': entry, 'tweet_id': subentry, 'tweet_text': data['text'].item(), 'tweet_class': gt_devB[subentry]})
        print(listB_dev[-1])
      elif subentry in gt_trainB:
        listB_train.append({'Journal': entry, 'tweet_id': subentry, 'tweet_text': data['text'].item(), 'tweet_class': gt_trainB[subentry]})
        print(listB_train[-1])
      else:
        print("ERROR: Class not found for a tweet")

    replies_path = os.path.join(data_source_path, subentry, "replies")
    for reply in os.listdir(replies_path):
      #print("Reply ID: ", reply[:-5])
      reply_id = str(reply[:-5])
      reply_path =  os.path.join(replies_path, reply)
      #print(reply_path)
      reply_data = pd.read_json(path_or_buf=reply_path, lines=True)
      if reply_id in gt_devA:
        listA_dev.append({'Journal': entry, 'tweet_id': subentry, 'tweet_text': listB_dev[-1]['tweet_text'], 'tweet_class': listB_dev[-1]['tweet_class'], 'reply_id': reply_id, 'reply_text': reply_data['text'].item(), 'reply_class':gt_devA[reply_id]})
        print(listA_dev[-1])
      elif reply_id in gt_trainA:
        listA_train.append({'Journal': entry, 'tweet_id': subentry, 'tweet_text': listB_train[-1]['tweet_text'], 'tweet_class': listB_train[-1]['tweet_class'], 'reply_id': reply_id, 'reply_text': reply_data['text'].item(), 'reply_class':gt_trainA[reply_id]})
        print(listA_train[-1])
      else:
        print("ERROR: Class not found for a tweet reply")

{'Journal': 'ebola-essien', 'tweet_id': '521346721226711040', 'tweet_text': 'Unconfirmed reports claim that Michael Essien has contracted Ebola. http://t.co/VASQrZdLhH', 'tweet_class': 'false'}
{'Journal': 'ebola-essien', 'tweet_id': '521346721226711040', 'tweet_text': 'Unconfirmed reports claim that Michael Essien has contracted Ebola. http://t.co/VASQrZdLhH', 'tweet_class': 'false', 'reply_id': '521368022880378882', 'reply_text': '@LobittaT @FootballcomEN my family are not african. Denied.', 'reply_class': 'comment'}
{'Journal': 'ebola-essien', 'tweet_id': '521346721226711040', 'tweet_text': 'Unconfirmed reports claim that Michael Essien has contracted Ebola. http://t.co/VASQrZdLhH', 'tweet_class': 'false', 'reply_id': '521368940669583360', 'reply_text': "@AyyBant @FootballcomEN Fuck off. You're an attention seeking cunt.", 'reply_class': 'comment'}
{'Journal': 'ebola-essien', 'tweet_id': '521346721226711040', 'tweet_text': 'Unconfirmed reports claim that Michael Essien has contracte

In [ ]:
dfA_train = pd.DataFrame(listA_train)
dfA_train.to_csv('replies.train.tsv', sep='\t', index=False)

dfA_dev = pd.DataFrame(listA_dev)
dfA_dev.to_csv('replies.dev.tsv', sep='\t', index=False)

dfB_train = pd.DataFrame(listB_train)
dfB_train.to_csv('tweets.train.tsv', sep='\t', index=False)

dfB_dev = pd.DataFrame(listB_dev)
dfB_dev.to_csv('tweets.dev.tsv', sep='\t', index=False)

## 2.2. Test

In [ ]:
ground_truth_path = 'semeval2017-task8-dataset/test/rumoureval2017-test'
gt_testA_path = os.path.join(ground_truth_path, 'subtaskA.json')
gt_testB_path = os.path.join(ground_truth_path, 'subtaskB.json')

In [ ]:
with open(gt_testA_path, 'r') as f:
    gt_testA = json.load(f)

with open(gt_testB_path, 'r') as f:
    gt_testB = json.load(f)

In [ ]:
test_data_path = 'semeval2017-task8-dataset/test/rumoureval2017-test/semeval2017-task8-test-data'

In [ ]:
listA_test = []
listB_test = []

for entry in os.listdir(test_data_path):
  data_source_path = os.path.join(test_data_path, entry)
  #print(data_source_path)
  tweet_path = os.path.join(data_source_path, "source-tweet")
  #print(tweet_path)
  for json_file in os.listdir(tweet_path):
    json_file_path = os.path.join(tweet_path, json_file)
    data = pd.read_json(path_or_buf=json_file_path, lines=True)
    entry = str(entry)
    if entry in gt_testB:
      listB_test.append({'tweet_id': entry, 'tweet_text': data['text'].item(), 'tweet_class': gt_testB[entry]})
      print(listB_test[-1])
    else:
      print("ERROR: Class not found for a tweet")

  replies_path = os.path.join(data_source_path, "replies")
  for reply in os.listdir(replies_path):
    reply_id = str(reply[:-5])
    reply_path =  os.path.join(replies_path, reply)
    reply_data = pd.read_json(path_or_buf=reply_path, lines=True)
    if reply_id in gt_testA:
      listA_test.append({'tweet_id': entry, 'tweet_text': listB_test[-1]['tweet_text'], 'tweet_class': listB_test[-1]['tweet_class'], 'reply_id': reply_id, 'reply_text': reply_data['text'].item(), 'reply_class':gt_testA[reply_id]})
      print(listA_test[-1])
    else:
      print("ERROR: Class not found for a tweet reply")

{'tweet_id': '524948206023880704', 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'tweet_class': 'true'}
{'tweet_id': '524948206023880704', 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'tweet_class': 'true', 'reply_id': '524950324604592128', 'reply_text': '“@CBCAlerts: #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial." @PoirierJeffrey U OK? 😦😨 #workfromhomesvp', 'reply_class': 'comment'}
{'tweet_id': '524948206023880704', 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'tweet_class': 'true', 'reply_id': '524950962373287936', 'reply_text': '@CBCAlerts @RepMattSalmon @CBCNews Obviously, I spoke too soon.  They got him.  Good job.  Hope the 

In [ ]:
dfA_test = pd.DataFrame(listA_test)
dfA_test.to_csv('replies.test.tsv', sep='\t', index=False)

dfB_test = pd.DataFrame(listB_test)
dfB_test.to_csv('tweets.test.tsv', sep='\t', index=False)

# 3. Load data

In [ ]:
# openai.api_key = os.getenv("") or ""

# openai.Engine.list()


In [ ]:
!ls

'CS 886.ipynb'	    replies.train.tsv   semeval2017-task8-dataset	    tweets.test.tsv
 replies.dev.tsv    replies.tsv         semeval2017-task8-dataset.tar.bz2   tweets.train.tsv
 replies.test.tsv   results	        tweets.dev.tsv			    tweets.tsv


In [ ]:
tweets_train = pd.read_csv('tweets.train.tsv', sep='\t')

In [ ]:
tweets_dev = pd.read_csv('tweets.dev.tsv', sep='\t')

In [ ]:
tweets_test = pd.read_csv('tweets.test.tsv', sep='\t')

In [ ]:
tweets_train

,Journal,tweet_id,tweet_text,tweet_class
0,ebola-essien,521346721226711040,Unconfirmed reports claim that Michael Essien ...,false
1,ebola-essien,521360486387175424,AC Milan midfielder Michael Essien has been di...,false
2,putinmissing,576323086888361984,This appears to be the original source of the ...,unverified
3,putinmissing,576829262927413248,Very good on #Putin coup by @CoalsonR: Three S...,unverified
4,putinmissing,577258317942149120,Was Vladimir Putin netralized by an internal c...,unverified
...,...,...,...,...
267,sydneysiege,544319832486064128,"Hostage situation erupts in Sydney cafe, Austr...",true
268,sydneysiege,544283772569788416,Live Updates: Siege in Sydney Cafe http://t.co...,true
269,sydneysiege,544305745416581120,Police stand guard as hostages are held in a c...,false
270,sydneysiege,544282227035869184,BREAKING NEWS: Gunmen take hostages at Sydney ...,true


In [ ]:
tweets_dev

,Journal,tweet_id,tweet_text,tweet_class
0,germanwings-crash,580331561398108160,1047 call: DGAC source says pilots called «urg...,false
1,germanwings-crash,581293286268129280,GERMAN NEWS: Co-Pilot of Germanwings Airbus Wa...,false
2,germanwings-crash,581386094337474560,RT @khjelmgaard: German media reporting #Andre...,unverified
3,germanwings-crash,580319078155468800,Germanwings Airbus A320 crashes in French Alps...,true
4,germanwings-crash,580360165540642816,@FRANCE24 reporter James André at French Civil...,false
5,germanwings-crash,580324027715063808,JUST IN: Germanwings plane crashes in southern...,true
6,germanwings-crash,580882341880446977,Report: Co-Pilot Locked Out Of Cockpit Before ...,true
7,germanwings-crash,580320684305416192,Accident aircraft looks to be Germanwings (Air...,unverified
8,germanwings-crash,580348081100734464,"#Germanwings ""sorry to confirm"" 144 passengers...",true
9,germanwings-crash,580325090367315968,"Flight #4U9525 initially climbed to 38,000 fee...",unverified


In [ ]:
tweets_test

,tweet_id,tweet_text,tweet_class
0,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true
1,524936793633083394,Very tense situation in Ottawa this morning. ...,true
2,544315472075042818,US consulate in #Sydney reportedly evacuated a...,unverified
3,581359544682614784,Germanwings co-pilot had serious depressive ep...,unverified
4,500280249629036544,Police name the officer who shot #Ferguson tee...,unverified
5,758159624122097664,Anyone who knows Marina Joyce personally knows...,false
6,763098277986209792,Hillary Mystery Handler Spotted With Diazepam ...,false
7,544274544174071809,Take a break from bashing Cheney and waterboar...,false
8,580352273001410560,NEWS: 'Emergency Emergency' was the final dist...,false
9,553561170637238272,BREAKING: Police order all shops closed in fam...,unverified


In [ ]:
# Inspect the nature of tweets in the dev set
for index, tweet in tweets_dev.iterrows():
  print(tweet['Journal'])
  print(tweet['tweet_text'])

germanwings-crash
1047 call: DGAC source says pilots called «urgence, urgence» (“emergency, emergency”), as opposed to a mayday call or 7700 squawk. #4U9525
germanwings-crash
GERMAN NEWS: Co-Pilot of Germanwings Airbus Was MUSLIM CONVERT. Can any muslim be trusted near an airplane now?
germanwings-crash
RT @khjelmgaard: German media reporting #AndreasLubitz had a 'serious depressive episode' six years ago http://t.co/MUdmHPz2ro #Germanwings
germanwings-crash
Germanwings Airbus A320 crashes in French Alps near Digne http://t.co/yNlWbNJmYI
germanwings-crash
@FRANCE24 reporter James André at French Civil Aviation Authority HQ: "#Germanwings plane did NOT report distress signal", no mayday.
germanwings-crash
JUST IN: Germanwings plane crashes in southern France, up to 150 feared dead http://t.co/GIZjXnqBU3
germanwings-crash
Report: Co-Pilot Locked Out Of Cockpit Before Fatal Plane Crash http://t.co/aXGQuacv2E #Germanwings http://t.co/T80L7BhXX6
germanwings-crash
Accident aircraft looks to 

# 4. Task B: Veracity Prediction

In [ ]:
openai.api_key = ""

Performed on Dev set to tune the hyperparameters, then the results are reported on the test set

In [ ]:
# journal = "germanwings-crash"
# text = "1047 call: DGAC source says pilots called «urgence, urgence» (“emergency, emergency”), as opposed to a mayday call or 7700 squawk. #4U9525"
# prompt = """Verify whether the rumourmous tweet is true, false or unverified based on the context below. Justify your choice.
#         Context: The tweet source is from {}
#         Rumorous tweet: {}
#         Answer:"""

# prompt = prompt.format(journal,text)
# res = openai.Completion.create(model="gpt-3.5-turbo", prompt=prompt, max_tokens=256) #engine='text-davinci-003'

In [ ]:
def predict(tweets,
            out_path,
            content,
            function,
            function_name,
            description,
            dump_file):
  matches = 0
  res_list = []
  for index, tweet in tweets.iterrows():
    content_filler = "from " + tweet['Journal'] if 'Journal' in tweet else ""
    filled_content = content.format(description, content_filler, tweet['tweet_text'])
    result = openai.ChatCompletion.create(
      model="gpt-4", #gpt-3.5-turbo
      temperature=0.8,
      messages=[{"role": "user", "content": filled_content}],
      functions=[function],
      function_call={"name": function_name},
    )
    prediction = json.loads(result["choices"][0]["message"]["function_call"]["arguments"])["prediction"][0]
    confidence = json.loads(result["choices"][0]["message"]["function_call"]["arguments"])["confidence"]
    justification = json.loads(result["choices"][0]["message"]["function_call"]["arguments"])["justification"]
    print(prediction)
    if (prediction.strip() == tweet['tweet_class'].strip()):
      matches += 1
    if 'Journal' in tweet:
      res_list.append({'Journal':tweet['Journal'], 'tweet_id': tweet['tweet_id'], 'tweet_text': tweet['tweet_text'], 'ground_truth': tweet['tweet_class'], 'prediction': prediction, 'confidence': confidence, 'justification': justification})
    else:
      res_list.append({'tweet_id': tweet['tweet_id'], 'tweet_text': tweet['tweet_text'], 'ground_truth': tweet['tweet_class'], 'prediction': prediction, 'confidence': confidence, 'justification': justification})
    print(res_list[-1])
    print(matches, "/", len(tweets))

    with open(dump_file, 'a') as outfile:
      json.dump(res_list[-1], outfile)
      outfile.write('\n')
  #res_df = pd.DataFrame(res_list)
  #res_df.to_csv(out_path, sep='\t', index=False)
  return matches #res_list

## 4.1. 3-class Predicition: True, False or Unverified

In [ ]:
rumour_veracity_prediction="""
true: if the tweet is verified to be true and factual from reliable crowdsourced data sources, even beyond the tweet text. \n
false: if the tweet is verified to be a false rumour or a misinformation, from any possible data sources. \n
unverified: if the tweet cannot be verified from data sources."""

function = {
   "name": "predict_tweet_class",
   "description": "Predict the veracity of a given tweet text, and Justify the prediction choice",
   "parameters": {
       "type": "object",
       "properties": {
           "prediction": {
               "type": "array",
               "items": {
                   "type": "string",
                   "enum": [
                       "true",
                       "false",
                       "unverified"
                   ]
               },
               "description": "The predicted tweet class."
           },
           "confidence":{
              "type": "number",
              "description": "The confidence value of the prediction judgment."
           },
           "justification": {
              "type": "string",
              "description": "The justification for which a class is selected."
           }
       },
       "required": [
           "prediction",
           "confidence",
           "justification",
       ]
   }
}

### 4.1.1. Few-shot learning

In [ ]:
subset = tweets_train[:20]
for x, y in zip(subset.tweet_text, subset.tweet_class):
  print("tweet: ", x)
  print("prediction: ", y)
  print('\n')

tweet:  Unconfirmed reports claim that Michael Essien has contracted Ebola. http://t.co/VASQrZdLhH
prediction:  false


tweet:  AC Milan midfielder Michael Essien has been diagnosed with Ebola. Get well soon Michael. [Daily Times] http://t.co/r6y8d9HMAw
prediction:  false


tweet:  This appears to be the original source of the Putin/Kabayeva/Swiss clinic rumour http://t.co/bKfJTSGdca
prediction:  unverified


tweet:  Very good on #Putin coup by @CoalsonR: Three Scenarios For A Succession In Russia http://t.co/fotdqxDfEV via @RFERL http://t.co/RdA0giizeb
prediction:  unverified


tweet:  Was Vladimir Putin netralized by an internal coup? Or maybe he's vacationing in Harper's closet:  http://t.co/tlr3cFLEyv #LookingForNarnia
prediction:  unverified


tweet:  Coup in #Russia? Good article by @Forbes. http://t.co/aaNCpb0blW #RamzanKadyrov #Putin #putindead http://t.co/OfivdTlTbO
prediction:  unverified


tweet:  Coup? RT @jimgeraghty: Rumors all Russian military attaches at embassy in Lond

In [ ]:
content_fs = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text between quotations "{}" to one category from [true, false, unverified].\n
Below is a description of each class: \n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice.

Below are 3 examples of tweets and their corresponding category predictions to learn from them: \n

tweet:  France: 10 people dead after shooting at HQ of satirical weekly newspaper #CharlieHebdo, according to witnesses http://t.co/FkYxGmuS58
prediction:  true

tweet:  Unformed Russian Embassy staff in London have left for Russia  Rumours Putin HAS DIED! http://t.co/zSIV8w6FJ2 via @ShaunyNews #PutinDead?
prediction:  false

tweet:  AFP reports there are 2 dead, and 5 hostages being held in the Kosher store in Eastern Paris; separate incident to charlie hebdo shooters.
prediction:  unverified
"""

In [ ]:
sum = 0

In [ ]:
sum += predict(tweets_test, '', content=content_fs, function=function, function_name="predict_tweet_class", description=rumour_veracity_prediction, dump_file="results/testB.3way.gpt4.fewshot_3exmp_temp0.8_2.jsonl")
sum

u
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'ground_truth': 'true', 'prediction': 'u', 'confidence': 0.6, 'justification': "The tweet is reporting on a breaking news event. While the source, CBCNews, is generally reliable, the information hasn't been independently verified. The information could change as more details emerge. Therefore, the tweet is categorised as unverified with a moderate confidence level."}
0 / 28
u
{'tweet_id': 524936793633083394, 'tweet_text': 'Very tense situation in Ottawa this morning.  Multiple gun shots fired outside of our caucus room.  I am safe and in lockdown. Unbelievable.', 'ground_truth': 'true', 'prediction': 'u', 'confidence': 0.6, 'justification': "While the tweet describes a tense situation which may be true, the specifics of the event such as 'multiple gunshots fired outside of our caucus room' cannot be 

0

### 4.1.2. One-shot Learning

In [ ]:
content_os = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text between quotations "{}" to one category from [true, false, unverified].\n
Below is a description of each class: \n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice.

Below is an example of a tweet and the corresponding category predictions to learn from it: \n

tweet:  France: 10 people dead after shooting at HQ of satirical weekly newspaper #CharlieHebdo, according to witnesses http://t.co/FkYxGmuS58
prediction:  true
"""

In [ ]:
sum += predict(tweets_test[5:], '', content=content_os, function=function, function_name="predict_tweet_class", description=rumour_veracity_prediction, dump_file="results/testB.3way.gpt4.fewshot_1exmp_temp0.8.jsonl")
sum

### 4.1.3. Zero-shot learning

In [ ]:
content = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text {} to a category.\n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice."""

In [ ]:
predict(tweets_test[22:], 'results/testB.tsv', content=content, function=function, function_name="predict_tweet_class", description=rumour_veracity_prediction, dump_file="results/testB.3way.gpt4_2.jsonl")

unverified
{'tweet_id': 775057555865206784, 'tweet_text': 'What do you think Hillary is suffering from?', 'ground_truth': 'false', 'prediction': 'unverified', 'confidence': 0.95, 'justification': 'The tweet is asking a question and does not present any factual claim or rumour that can be verified as true or false'}
0 / 6
unverified
{'tweet_id': 768859780240773121, 'tweet_text': 'Sick Hillary is seen here using great caution as she attempts to get out of a car. #HillarysHealth https://t.co/1ys6vMJrds', 'ground_truth': 'false', 'prediction': 'unverified', 'confidence': 0.6, 'justification': "The tweet is making a claim about a person's health which is a sensitive topic. Without reliable sources or confirmed reports, it's difficult to verify the truthfulness of such a claim. Additionally, the language used in the tweet appears biased and may not represent factual information. So, it's safe to categorize this tweet as unverified."}
0 / 6
unverified
{'tweet_id': 553553288625672192, 'tweet_t

1

## 4.2. 2-way Class Predicition: True or False

In [ ]:
rumour_veracity_prediction_2way="""
true: if the tweet is verified to be true and factual from reliable crowdsourced data sources, even beyond the tweet text. \n
false: if the tweet is verified to be a false rumour or a misinformation, from any possible data source. \n"""

function_2way = {
   "name": "predict_tweet_class_2way",
   "description": "Predict the veracity of a given tweet text, and Justify the prediction choice",
   "parameters": {
       "type": "object",
       "properties": {
           "prediction": {
               "type": "array",
               "items": {
                   "type": "string",
                   "enum": [
                       "true",
                       "false",
                   ]
               },
               "description": "The predicted tweet class."
           },
           "confidence":{
              "type": "number",
              "description": "The confidence value of the prediction judgment."
           },
           "justification": {
              "type": "string",
              "description": "The justification for which a class is selected."
           }
       },
       "required": [
           "prediction",
           "confidence",
           "justification",
       ]
   }
}

### 4.2.1. Few-shot Learning

In [ ]:
content_fs = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text between quotations "{}" to one category from [true, false].\n
Below is a description of each class: \n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice.

Below are few examples of tweets and their corresponding category predictions to learn from them: \n

tweet:  French police publishes photos of suspect in yesterday's Montrouge shooting. Maybe the same persons in Kosher market http://t.co/j5nQIl4Ytu
prediction:  true

tweet:  AC Milan midfielder Michael Essien has been diagnosed with Ebola. Get well soon Michael. [Daily Times] http://t.co/r6y8d9HMAw
prediction:  false

tweet:  France: 10 people dead after shooting at HQ of satirical weekly newspaper #CharlieHebdo, according to witnesses http://t.co/FkYxGmuS58
prediction:  true

tweet:  Unformed Russian Embassy staff in London have left for Russia  Rumours Putin HAS DIED! http://t.co/zSIV8w6FJ2 via @ShaunyNews #PutinDead?
prediction:  false
"""

In [ ]:
predict(tweets_test, '', content=content_fs, function=function_2way, function_name="predict_tweet_class_2way", description=rumour_veracity_prediction_2way, dump_file="results/testB.2way.gpt4.fewshot_4exmp_temp0.8.jsonl")

true
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'ground_truth': 'true', 'prediction': 'true', 'confidence': 0.9, 'justification': 'The tweet is from @CBCNews, a reliable news source, and covers a major event (the Ottawa shootings). The details provided in the tweet are factual and the tweet contains a link to the news article on their website for further information.'}
1 / 28
true
{'tweet_id': 524936793633083394, 'tweet_text': 'Very tense situation in Ottawa this morning.  Multiple gun shots fired outside of our caucus room.  I am safe and in lockdown. Unbelievable.', 'ground_truth': 'true', 'prediction': 'true', 'confidence': 0.9, 'justification': 'The tweet is reporting a specific event and situation (gun shots and lockdown in Ottawa), and it seems to be from a first-hand source who is in the middle of the event. Therefore, it is likely to be

17

### 4.2.2. One-shot Learning

In [ ]:
content_os = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text between quotations "{}" to one category from [true, false].\n
Below is a description of each class: \n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice.

Below is an example of a tweet and the corresponding category predictions to learn from it: \n

tweet:  AC Milan midfielder Michael Essien has been diagnosed with Ebola. Get well soon Michael. [Daily Times] http://t.co/r6y8d9HMAw
prediction:  false
"""

In [ ]:
predict(tweets_test, '', content=content_os, function=function_2way, function_name="predict_tweet_class_2way", description=rumour_veracity_prediction_2way, dump_file="results/testB.2way.gpt4_oneshot_temp0.8_4.jsonl")

true
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'ground_truth': 'true', 'prediction': 'true', 'confidence': 0.8, 'justification': 'The tweet is referring to a major news event reported by a reputable news source (@CBCNews). This increases the likelihood of the information being true. However, without cross-verification from other sources, the confidence is not 100%.'}
1 / 28
true
{'tweet_id': 524936793633083394, 'tweet_text': 'Very tense situation in Ottawa this morning.  Multiple gun shots fired outside of our caucus room.  I am safe and in lockdown. Unbelievable.', 'ground_truth': 'true', 'prediction': 'true', 'confidence': 0.9, 'justification': "The tweet seems to come from a direct witness of the event described, which could be verifiable from news reports of the incident. The user mentions being in 'lockdown', which aligns with standard sa

16

### 4.2.3. Zero-shot Learning

In [ ]:
content = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text {} to a category.\n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice."""

In [ ]:
sum = 0
sum+= predict(tweets_test, out_path='results/2way.testB.tsv', content=content, function=function_2way, function_name="predict_tweet_class_2way", description=rumour_veracity_prediction_2way, dump_file="results/testB.2way.gpt3.5_3.jsonl")
sum

t
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'ground_truth': 'true', 'prediction': 't', 'confidence': 0.9, 'justification': 'The tweet is from a reliable news source, CBC News, and provides information about a recent event. Additionally, the hashtag #Ottawa shootings suggests that the tweet is related to a real incident. Overall, the tweet text and the source credibility indicate that the information is likely to be true.'}
0 / 28
t
{'tweet_id': 524936793633083394, 'tweet_text': 'Very tense situation in Ottawa this morning.  Multiple gun shots fired outside of our caucus room.  I am safe and in lockdown. Unbelievable.', 'ground_truth': 'true', 'prediction': 't', 'confidence': 0.9, 'justification': 'The tweet mentions multiple gun shots fired outside of the caucus room in Ottawa and the person is in lockdown. This indicates a tense and dangerous

3

## 4.3. 2-way (Verified/Unverified) + 2-way (True/False) Classification

In [ ]:
rumour_verification="""
verified: if the tweet information can be verified to be true or verified to be false in any data sources. \n
unverified: if the tweet is a rumour that cannot be verified from any data sources. \n"""

function_2way_2way = {
   "name": "predict_tweet_class_2way_2way",
   "description": "Predict the veracity of a given tweet text, and Justify the prediction choice",
   "parameters": {
       "type": "object",
       "properties": {
           "prediction": {
               "type": "array",
               "items": {
                   "type": "string",
                   "enum": [
                       "verified",
                       "unverified",
                   ]
               },
               "description": "The predicted tweet class."
           },
           "confidence":{
              "type": "number",
              "description": "The confidence value of the prediction judgment."
           },
           "justification": {
              "type": "string",
              "description": "The justification for which a class is selected."
           }
       },
       "required": [
           "prediction",
           "confidence",
           "justification",
       ]
   }
}

### 4.3.1. Few-shot Learning

In [ ]:
content_fs = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text between quotations "{}" to one category from [verified, unverified].\n
Below is a description of each class: \n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice.

Below are few examples of tweets and their corresponding category predictions to learn from them: \n

tweet:  AC Milan midfielder Michael Essien has been diagnosed with Ebola. Get well soon Michael. [Daily Times] http://t.co/r6y8d9HMAw
prediction:  verified

tweet:  Putin reappears on television amid rumours his 'girlfriend' is about 2 give birth http://t.co/yJuN1F0QXE http://t.co/B7FXmo03jK @Scofield_Fx
prediction:  unverified
"""

In [ ]:
predict(tweets_test, out_path='', content=content_fs, function=function_2way_2way, function_name="predict_tweet_class_2way_2way", description=rumour_verification, dump_file="results/testB.2by2way.gpt4.fewshot-2exmpl_x.jsonl")

verified
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'ground_truth': 'true', 'prediction': 'verified', 'confidence': 0.9, 'justification': 'The tweet is quoting a news story from a reliable source (@CBCNews), which implies that the information has been verified by this source. The hashtags and link to the news story also add credibility to the tweet.'}
0 / 28
unverified
{'tweet_id': 524936793633083394, 'tweet_text': 'Very tense situation in Ottawa this morning.  Multiple gun shots fired outside of our caucus room.  I am safe and in lockdown. Unbelievable.', 'ground_truth': 'true', 'prediction': 'unverified', 'confidence': 0.7, 'justification': 'The tweet talks about a live situation that is happening right now which makes it hard to verify at the moment of the tweet. Therefore, it is classified as unverified. However, the confidence is not too h

5

### 4.3.2. One-shot Learning

In [ ]:
content_os = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text between quotations "{}" to one category from [verified, unverified].\n
Below is a description of each class: \n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice.

Below is an example of a tweet and the corresponding category predictions to learn from it: \n

tweet:  AC Milan midfielder Michael Essien has been diagnosed with Ebola. Get well soon Michael. [Daily Times] http://t.co/r6y8d9HMAw
prediction:  verified
"""

In [ ]:
predict(tweets_test, out_path='', content=content_os, function=function_2way_2way, function_name="predict_tweet_class_2way_2way", description=rumour_verification, dump_file="results/testB.2by2way.gpt4.oneshot_x.jsonl")

verified
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'ground_truth': 'true', 'prediction': 'verified', 'confidence': 0.85, 'justification': 'The tweet is from a verified news source (@CBCNews), which generally provides accurate news updates. The usage of specific location references (#Ottawa, Parliament, War Memorial) and a link to the news story also lend credibility to the tweet. However, the confidence is not 1 as it is always possible that the information could be incorrect or misleading.'}
0 / 28
unverified
{'tweet_id': 524936793633083394, 'tweet_text': 'Very tense situation in Ottawa this morning.  Multiple gun shots fired outside of our caucus room.  I am safe and in lockdown. Unbelievable.', 'ground_truth': 'true', 'prediction': 'unverified', 'confidence': 0.7, 'justification': 'The tweet reports a serious incident but does not provide a

7

### 4.3.3. Zero-shot Learning

In [ ]:
content = """###Rumour Veracity Prediction###\n
{}\n\n
###Instructions###\n
Categorise the following tweet text {} to a category.\n
{}.\n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice."""

In [ ]:
sum = 0

In [ ]:
sum+= predict(tweets_test, out_path='results/2by2way-verfunverf.testB.tsv', content=content, function=function_2way_2way, function_name="predict_tweet_class_2way_2way", description=rumour_verification, dump_file="results/testB.2by2way.gpt4_x.jsonl")

verified
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'ground_truth': 'true', 'prediction': 'verified', 'confidence': 0.9, 'justification': 'This tweet is from @CBCNews - a well-known news organization, which lends credibility to the information. The tweet is also about a very specific event that can be cross-checked with other news sources for verification.'}
0 / 28
unverified
{'tweet_id': 524936793633083394, 'tweet_text': 'Very tense situation in Ottawa this morning.  Multiple gun shots fired outside of our caucus room.  I am safe and in lockdown. Unbelievable.', 'ground_truth': 'true', 'prediction': 'unverified', 'confidence': 0.6, 'justification': "The tweet describes a situation that could be possible, but there is no specific data or source provided to verify the truth of the situation. Furthermore, the use of a personal perspective and the

In [ ]:
verified_list = [{'tweet_id': 580352273001410560, 'tweet_text': "NEWS: 'Emergency Emergency' was the final distress call that was sent from the cockpit of flight #4U9525  - @PollyR_Aviation", 'ground_truth': 'false', 'prediction': 'verified', 'confidence': 0.9, 'justification': 'The tweet mentions a distress call from the cockpit of a specific flight, and it includes the handle of a verified aviation account (@PollyR_Aviation). This suggests that the tweet has been verified from a reliable source and provides specific information about the incident.'},
{'tweet_id': 500298588992593920, 'tweet_text': "Police have named the cop who shot Michael Brown. @alicesperi's latest report from #Ferguson: http://t.co/1K79nsOtx3 http://t.co/cV3VCkUR6L", 'ground_truth': 'unverified', 'prediction': 'verified', 'confidence': 0.9, 'justification': "The tweet provides a link to a report from a reliable source, indicating that the information has been verified. Additionally, the mention of the cop's name suggests that this information has been confirmed."},
{'tweet_id': 524941720249978880, 'tweet_text': 'Footage from Globe reporter captures exchange of gun fire in Parliament Hill building https://t.co/FOu4PbNCDq', 'ground_truth': 'true', 'prediction': 'verified', 'confidence': 0.8, 'justification': 'The tweet mentions that the footage is from a Globe reporter, indicating that it has been verified. Additionally, the mention of an exchange of gunfire in the Parliament Hill building is a significant event that would likely be covered by multiple news sources, further supporting the verification of the tweet.'},
{'tweet_id': 524961721744900097, 'tweet_text': "Canadian soldier who was shot at #Ottawa's War Memorial is a reservist serving with the Argyll and Sutherland Highlanders in #Hamilton, Ont.", 'ground_truth': 'true', 'prediction': 'verified', 'confidence': 0.8, 'justification': 'The tweet mentions specific details about the Canadian soldier, including the location and the unit they serve with. This information can be verified through official sources or news reports.'}]

verified_tweets = pd.DataFrame(verified_list)

In [ ]:
verified_tweets = verified_tweets.rename(columns={"ground_truth": "tweet_class"})

In [ ]:
sum+= predict(verified_tweets, out_path='results/2by2way-truefalse.testB.tsv', content=content, function=function_2way, function_name="predict_tweet_class_2way", description=rumour_veracity_prediction_2way)

t
{'tweet_id': 580352273001410560, 'tweet_text': "NEWS: 'Emergency Emergency' was the final distress call that was sent from the cockpit of flight #4U9525  - @PollyR_Aviation", 'ground_truth': 'false', 'prediction': 't', 'confidence': 0.8, 'justification': 'The tweet text mentions a distress call from the cockpit of flight #4U9525. If this distress call is verified to be true and factual from reliable crowdsourced data sources, it can be categorized as a true news. However, further investigation is needed to confirm the veracity of the distress call.'}
t
{'tweet_id': 500298588992593920, 'tweet_text': "Police have named the cop who shot Michael Brown. @alicesperi's latest report from #Ferguson: http://t.co/1K79nsOtx3 http://t.co/cV3VCkUR6L", 'ground_truth': 'unverified', 'prediction': 't', 'confidence': 0.9, 'justification': 'The tweet text indicates that the police have named the cop who shot Michael Brown, which is a factual statement. Additionally, the tweet includes a link to a repo

# 5. Task A: SDQC Stance Classification

In [ ]:
openai.api_key = ""

In [ ]:
replies_test = pd.read_csv('replies.test.tsv', sep='\t')

In [ ]:
replies_test

,tweet_id,tweet_text,tweet_class,reply_id,reply_text,reply_class
0,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524950324604592128,“@CBCAlerts: #Ottawa shootings. Gunman killed ...,comment
1,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524950962373287936,"@CBCAlerts @RepMattSalmon @CBCNews Obviously, ...",comment
2,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524952544595177474,@CBCAlerts @PnPCBC @CBCNews Round up these ass...,comment
3,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524951904212615168,@CBCAlerts tragic. Stay safe our law informers...,comment
4,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524997088325955584,@CBCAlerts @ImStillSassy @CBCNews There are to...,comment
...,...,...,...,...,...,...
1016,769988636754505729,267 days since Sick Hillary had a press confer...,false,770038900349923333,@PrisonPlanet #WheresHillary 😴 or 🍹 https://t....,comment
1017,769988636754505729,267 days since Sick Hillary had a press confer...,false,770035867381600260,@PrisonPlanet @BigKahuna919191 read my blog an...,comment
1018,769988636754505729,267 days since Sick Hillary had a press confer...,false,770041632360112130,@dystopia47 @PrisonPlanet https://t.co/1WFZBr3aiB,comment
1019,769988636754505729,267 days since Sick Hillary had a press confer...,false,769996788174561282,@PrisonPlanet \n#HideawayHillary,comment


In [ ]:
stance_content = """###Rumour SDQC Stance Classification has the following stance categories: ###\n
{}\n\n
###Instructions###\n
Categorise the following tweet reply {} \n
"{}" \n
to one stance category based on the following source tweet \n
"{}". \n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice."""

In [ ]:
def predict_stance(tweets,
            out_path,
            content,
            function,
            function_name,
            description,
            dump_file='results/4way.testA.jsonl',
            multiclass=False):
  matches = 0
  res_list = []
  cnt = 0

  for index, tweet in tweets.iterrows():
    print("Count: ", cnt)
    cnt += 1
    journal_info = "from " + tweet['Journal'] if 'Journal' in tweet else ""
    filled_content = content.format(description, journal_info, tweet['reply_text'], tweet['tweet_text'])
    result = openai.ChatCompletion.create(
      model="gpt-3.5-turbo", #"gpt-3.5-turbo"
      temperature=0.8,
      messages=[{"role": "user", "content": filled_content}],
      functions=[function],
      function_call={"name": function_name},
    )
    prediction = json.loads(result["choices"][0]["message"]["function_call"]["arguments"])["prediction"][0]
    confidence = json.loads(result["choices"][0]["message"]["function_call"]["arguments"])["confidence"]
    if multiclass:
      preds = json.loads(result["choices"][0]["message"]["function_call"]["arguments"])["prediction"]
      prediction = preds[0]

      confs = json.loads(result["choices"][0]["message"]["function_call"]["arguments"])["confidence"]
      confidence = confs[0]

      print(preds)
      print (confs)

    justification = json.loads(result["choices"][0]["message"]["function_call"]["arguments"])["justification"]


    if (prediction.strip() == tweet['reply_class'].strip()):
      matches += 1

    jsonl_row = {}
    if 'Journal' in tweet:
      jsonl_row = {'Journal':tweet['Journal'], 'tweet_id': tweet['tweet_id'], 'tweet_text': tweet['tweet_text'], 'tweet_class': tweet['tweet_class'], 'reply_id': tweet['reply_id'], 'reply_text': tweet['reply_text'], 'ground_truth': tweet['reply_class'],'prediction': prediction, 'confidence': confidence, 'justification': justification}
    else:
       jsonl_row = {'tweet_id': tweet['tweet_id'], 'tweet_text': tweet['tweet_text'], 'tweet_class': tweet['tweet_class'], 'reply_id': tweet['reply_id'], 'reply_text': tweet['reply_text'], 'ground_truth': tweet['reply_class'], 'prediction': prediction, 'confidence': confidence, 'justification': justification}
    res_list.append(jsonl_row)

    print(jsonl_row)
    print(matches, "/", len(tweets))

    with open(dump_file, 'a') as outfile:
      json.dump(res_list[-1], outfile)
      outfile.write('\n')

  #res_df = pd.DataFrame(res_list)
  #res_df.to_csv(out_path, sep='\t', index=False)
  return matches #res_list

## 5.1. 4-way classification

In [ ]:
tweet_reply_stance_description="""
support: the author of the response supports the veracity of the rumour represented in the source tweet.\n
deny: the author of the response denies the veracity of the rumour represented in the source tweet.\n
query: the author of the response asks for additional evidence in relation to the veracity of the rumour.\n
comment: the author of the response makes their own comment without a clear contribution to assessing the veracity of the rumour."""

stance_function = {
   "name": "predict_reply_stance",
   "description": "SDQC stands for Support/Deny/Query/Comment. Predict the SDQC stance of a given tweet text, and Justify the prediction choice",
   "parameters": {
       "type": "object",
       "properties": {
           "prediction": {
               "type": "array",
               "items": {
                   "type": "string",
                   "enum": [
                       "support",
                       "deny",
                       "query",
                       "comment",
                   ]
               },
               "description": "The predicted reply stance."
           },
           "confidence":{
              "type": "array",
              "items":{
                  "type": "number",
                  "description": "The confidence value of the prediction judgment.",
              }
           },
           "justification": {
              "type": "string",
              "description": "The justification for which a stance is selected."
           }
       },
       "required": [
           "prediction",
           "confidence",
           "justification",
       ]
   }
}

In [ ]:
#  gpt-3.5-turbo 594 /1021
#  gpt-4 634/1021
predict_stance(replies_test, "results/4way.gpt4.testA.tsv", stance_content, stance_function, 'predict_reply_stance', tweet_reply_stance_description)

query
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'tweet_class': 'true', 'reply_id': 524950324604592128, 'reply_text': '“@CBCAlerts: #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial." @PoirierJeffrey U OK? 😦😨 #workfromhomesvp', 'ground_truth': 'comment', 'prediction': 'query', 'confidence': 0.9, 'justification': 'The user is asking another user (@PoirierJeffrey) if they are okay, which implies that they are querying for more information or clarification about the situation, rather than outright supporting, denying, or commenting on the event itself.'}
0 / 1021
support
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'tweet_class': 'true', 'reply_id': 524950962373287936, 'reply_t

634

##5.2. 3-way classification

In [ ]:
replies_test_no_comment = replies_test[replies_test.reply_class != 'comment']

In [ ]:
len(replies_test_no_comment)

243

In [ ]:
replies_test_no_comment

,tweet_id,tweet_text,tweet_class,reply_id,reply_text,reply_class
7,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524978844155912193,@CBCAlerts @CBCNews whats happening at the Ott...,query
9,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524999801973506048,@Trial_Watcher1 @CBCAlerts @CBCNews And the ne...,deny
12,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524948710942588928,@CBCAlerts @CBCNews any word on the condition ...,query
13,524948206023880704,The latest @CBCNews on #Ottawa shootings. Gunm...,true,524999012538408963,@ImStillSassy @CBCAlerts @CBCNews One parelent...,support
24,524936793633083394,Very tense situation in Ottawa this morning. ...,true,524937839960928256,MP “@markstrahl: Multiple gun shots fired outs...,support
...,...,...,...,...,...,...
955,769988636754505729,267 days since Sick Hillary had a press confer...,false,769996635514474497,@PrisonPlanet @LindaSuhler wait.. Wasn't Kimme...,query
968,769988636754505729,267 days since Sick Hillary had a press confer...,false,770074745823985669,@PrisonPlanet will she make 365? Will she ever...,query
1003,769988636754505729,267 days since Sick Hillary had a press confer...,false,769989142075809792,"@PrisonPlanet \n\nProbably another ""fallacious...",deny
1005,769988636754505729,267 days since Sick Hillary had a press confer...,false,769993008896995328,@PrisonPlanet I believe they will rig this ele...,support


In [ ]:
tweet_reply_stance_description_sdq="""
support: the author of the response supports the veracity of the rumour represented in the source tweet.\n
deny: the author of the response denies the veracity of the rumour represented in the source tweet.\n
query: the author of the response asks for additional evidence in relation to the veracity of the rumour.\n
"""
stance_function_sdq = {
   "name": "predict_reply_stance_sdq",
   "description": "SDQ stands for Support/Deny/Query. Predict the SDQ stance of a given tweet text, and Justify the prediction choice",
   "parameters": {
       "type": "object",
       "properties": {
           "prediction": {
               "type": "array",
               "items": {
                   "type": "string",
                   "enum": [
                       "support",
                       "deny",
                       "query",
                   ]
               },
               "description": "The predicted reply stance."
           },
           "confidence":{
              "type": "number",
              "description": "The confidence value of the prediction judgment."
           },
           "justification": {
              "type": "string",
              "description": "The justification for which a stance is selected."
           }
       },
       "required": [
           "prediction",
           "confidence",
           "justification",
       ]
   }
}

In [ ]:
# GPT-4 : 188/243
# GPT-3.5: 158/243
predict_stance(replies_test_no_comment[50:], "", stance_content, stance_function_sdq, 'predict_reply_stance_sdq', tweet_reply_stance_description_sdq, dump_file='results/testA.3way.gpt3.5.jsonl')

Count:  0
{'tweet_id': 758159624122097664, 'tweet_text': 'Anyone who knows Marina Joyce personally knows she has a serious drug addiction. she needs help, but in the form of rehab #savemarinajoyce', 'tweet_class': 'false', 'reply_id': 758296316648849408, 'reply_text': '@sampepper who are u', 'ground_truth': 'query', 'prediction': 'query', 'confidence': 0.8, 'justification': 'The tweet reply is asking for additional evidence or clarification about the drug addiction claim made in the source tweet.'}
1 / 193
Count:  1
{'tweet_id': 758159624122097664, 'tweet_text': 'Anyone who knows Marina Joyce personally knows she has a serious drug addiction. she needs help, but in the form of rehab #savemarinajoyce', 'tweet_class': 'false', 'reply_id': 758480308471734277, 'reply_text': "@sampepper didn't you rape her?", 'ground_truth': 'query', 'prediction': 'deny', 'confidence': 0.9, 'justification': 'The tweet reply denies the veracity of the rumour by questioning the accusation made in the source t

120

## 5.3. Multiclass classification

In [ ]:
stance_content_multi = """###Rumour SDQC Stance Classification has the following stance categories: ###\n
{}\n\n
###Instructions###\n
Categorise the following tweet reply {} \n
"{}" \n
to one or more stance categories based on the following source tweet \n
"{}". \n
Give a confidence score between 0 and 1 for the predicted value. \n
And justify the prediction choice."""

In [ ]:
predict_stance(replies_test[:10], "results/multi.gpt3.testA.tsv", stance_content_multi, stance_function, 'predict_reply_stance', tweet_reply_stance_description, multiclass=True)

['comment']
[0.8]
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'tweet_class': 'true', 'reply_id': 524950324604592128, 'reply_text': '“@CBCAlerts: #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial." @PoirierJeffrey U OK? 😦😨 #workfromhomesvp', 'ground_truth': 'comment', 'prediction': 'comment', 'confidence': 0.8, 'justification': "The tweet reply does not explicitly support, deny, or query the veracity of the rumour. Instead, it is a comment expressing concern for the well-being of the user mentioned in the reply. Therefore, the reply falls under the 'comment' category."}
1 / 10
['comment']
[0.8]
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'tweet_class': 'true', 'reply_id': 524

6

In [ ]:
predict_stance(replies_test[:10], "results/multi.gpt4.testA.tsv", stance_content_multi, stance_function, 'predict_reply_stance', tweet_reply_stance_description)

['query']
[0.85]
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTSCe #cdnpoli', 'tweet_class': 'true', 'reply_id': 524950324604592128, 'reply_text': '“@CBCAlerts: #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial." @PoirierJeffrey U OK? 😦😨 #workfromhomesvp', 'ground_truth': 'comment', 'prediction': 'query', 'confidence': 0.85, 'justification': "The author of the reply tweet is asking another user '@PoirierJeffrey' if they're okay, indicating a query or request for information in response to the source tweet about the Ottawa shootings. The reply doesn't clearly support, deny, or comment on the veracity of the rumour in the source tweet."}
0 / 10
['support']
[0.85]
{'tweet_id': 524948206023880704, 'tweet_text': 'The latest @CBCNews on #Ottawa shootings. Gunman killed inside Parliament, soldier shot at War Memorial. http://t.co/kngapKTS

6

In [ ]:
predict_stance(replies_test[50:60], "results/multi.gpt4.testA.tsv", stance_content_multi, stance_function, 'predict_reply_stance', tweet_reply_stance_description)

['comment']
[1]
{'tweet_id': 544315472075042818, 'tweet_text': 'US consulate in #Sydney reportedly evacuated amid ongoing hostage situation at chocolate shop http://t.co/n4D3yGjso9 http://t.co/ayL7vvqxl8', 'tweet_class': 'unverified', 'reply_id': 544582023956815873, 'reply_text': '@mattbrisbon shut up you dumb cunt', 'ground_truth': 'comment', 'prediction': 'comment', 'confidence': 1, 'justification': 'The reply tweet contains an insult directed at a user without any clear relation to the content of the source tweet. It does not support, deny, or query the information in the source tweet, hence it is classified as a comment.'}
1 / 10
['comment']
[0.9]
{'tweet_id': 544315472075042818, 'tweet_text': 'US consulate in #Sydney reportedly evacuated amid ongoing hostage situation at chocolate shop http://t.co/n4D3yGjso9 http://t.co/ayL7vvqxl8', 'tweet_class': 'unverified', 'reply_id': 544365515255599104, 'reply_text': "@AztlanConnect we'll use the didgeridoos for cover", 'ground_truth': 'comm

7

In [ ]:
# import openai

# openai.api_key = ""

# content = """Classes: [true, false, unverfied]
# Rumourous Text: 1047 call: DGAC source says pilots called «urgence, urgence» (“emergency, emergency”), as opposed to a mayday call or 7700 squawk. #4U9525.

# Classify the rumorous tweet text from germanwings-crash into one of the above classes based on its veracity. true if the rumour is true. false if the rumour is baseless. unverfied if there is not enough information"""

# openai.ChatCompletion.create(
#   model="gpt-3.5-turbo",
#   temperature=0.9,
#   messages=[
#     {"role": "user", "content": content},
#   ]
# )


<OpenAIObject chat.completion id=chatcmpl-8OTfSrwOMF5mlZI6TaNFUzYXruEBS at 0x78c90b90d350> JSON: {
  "id": "chatcmpl-8OTfSrwOMF5mlZI6TaNFUzYXruEBS",
  "object": "chat.completion",
  "created": 1700844606,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Based on the information given, the rumorous tweet text from germanwings-crash cannot be classified into one of the above classes based on its veracity. The tweet text is unverified because there is not enough information to determine if the rumour is true or false."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 119,
    "completion_tokens": 55,
    "total_tokens": 174
  }
}